# Lab 9: Remote sensing of Surface Water / Floods

**Purpose:** The purpose of this lab is to demonstrate, step-by-step, the implementation of an efficient and robust approach for mapping surface water. You will also learn how the extracted surface water information can be used in conjunction with historical surface water information to extract flooded areas.

In [ ]:
%pylab inline

In [ ]:
# import ee api and geemap package
import ee
import math
import geemap
import pandas as pd
from geemap import colormaps as cmaps

In [ ]:
# try to initalize an ee session
# if not authenticated then run auth workflow and initialize
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

## Background

An efficient method for mapping water is image thresholding ([Schumann et al. 2009](https://doi.org/10.1109/TGRS.2009.2017937)). There are numerous methods for automated image thresholding, however, a popular method is the Otsu’s method ([Otsu, 1979](https://cw.fel.cvut.cz/b201/_media/courses/a6m33bio/otsu.pdf)). Otsu’s method is a histogram-based thresholding approach where the inter-class variance between two classes, a foreground and background class, is maximized. As stated, this approach assumes only two classes are present within an image which is rarely the case. Therefore, methods have been developed (i.e. [Donchyts et al. 2016](https://doi.org/10.3390/rs8050386) and [Cao et al. 2019](https://doi.org/10.3390/w11040786)) to constrain histogram sampling to areas that are more likely to represent a bimodal histogram of water/no water. The constrained histogram provides a more accurate estimation of a water threshold while not sacrificing the computation efficiency of the Otsu thresholding method.


## Surface water mapping

To begin we will start with accessing Sentinel-1 data. We will focus our analysis on Southeast Asia which experiences yearly flooding and has plenty of good test cases. To do this, we will assign the Sentinel-1 collection to a variable and filter by space, time, and metadata properties to get our image for processing:


### Pre-processing

As we explored in [Lab 6](https://colab.research.google.com/drive/1EzuXVxvlkdVPRwS9zbrMFEF_sAcKzvzb?usp=sharing), the SAR data on Earth Engine requires pre-processing which includes radiometric terrain correction and speckle filtering. Here we will define our functions to process the data.

In [ ]:
# function to convert power to dB units
def power_to_db(image):
    return ee.Image.constant(10).multiply(image.log10())

In [ ]:
def slope_correction(image):
    """This function applies the slope correction on a Sentinel-1 image.
    Function based on https:# doi.org/10.3390/rs12111867.
    Adapted from https:# github.com/ESA-PhiLab/radiometric-slope-correction/blob/master/notebooks/1%20-%20Generate%20Data.ipynb

    args:
        image (ee.Image): Sentinel-1 to perform correction on
        elevation (ee.Image): Input DEM to calculate slope corrections from
        model (str, optional): physical reference model to be applied. Options are 'volume' or 'surface'.
            default = volume
        buffer (int, optional): buffer in meters for layover/shadow mask. If zero then no buffer will be applied. default = 0
        scale (int, optional): reduction scale to process satellite heading compared to ground. Increasing will reduce
            chance of OOM errors but reduce local scale correction accuracy. default = 1000

    returns:
        ee.Image: slope corrected SAR imagery with look and local incidence angle bands

    raises:
        NotImplementedError: when keyword model is not of 'volume' or 'surface'
    """

    def _volumetric_model_SCF(theta_iRad, alpha_rRad):
        """Closure funnction for calculation of volumetric model SCF

        args:
            theta_iRad (ee.Image): incidence angle in radians
            alpha_rRad (ee.Image): slope steepness in range

        returns:
            ee.Image
        """

        # model
        nominator = (ninetyRad.subtract(theta_iRad).add(alpha_rRad)).tan()
        denominator = (ninetyRad.subtract(theta_iRad)).tan()
        return nominator.divide(denominator)

    def _surface_model_SCF(theta_iRad, alpha_rRad, alpha_azRad):
        """Closure funnction for calculation of direct model SCF

        args:
            theta_iRad (ee.Image): incidence angle in radians
            alpha_rRad (ee.Image): slope steepness in range
            alpha_azRad (ee.Image): slope steepness in azimuth

        returns:
            ee.Image
        """

        # model
        nominator = (ninetyRad.subtract(theta_iRad)).cos()
        denominator = alpha_azRad.cos().multiply(
            (ninetyRad.subtract(theta_iRad).add(alpha_rRad)).cos()
        )

        return nominator.divide(denominator)

    def _erode(image, distance):
        """Closure function to buffer raster values

        args:
            image (ee.Image): image that should be buffered
            distance (int): distance of buffer in meters

        returns:
            ee.Image
        """

        d = (
            image.Not()
            .unmask(1)
            .fastDistanceTransform(10)
            .sqrt()
            .multiply(ee.Image.pixelArea().sqrt())
        )

        return image.updateMask(d.gt(distance))

    def _masking(alpha_rRad, theta_iRad, buffer):
        """Closure function for masking of layover and shadow

        args:
            alpha_rRad (ee.Image): slope steepness in range
            theta_iRad (ee.Image): incidence angle in radians
            buffer (int): buffer in meters

        returns:
            ee.Image
        """
        # layover, where slope > radar viewing angle
        layover = alpha_rRad.lt(theta_iRad).rename("layover")

        # shadow
        shadow = alpha_rRad.gt(
            ee.Image.constant(-1).multiply(ninetyRad.subtract(theta_iRad))
        ).rename("shadow")

        # add buffer to layover and shadow
        if buffer > 0:
            layover = _erode(layover, buffer)
            shadow = _erode(shadow, buffer)

        # combine layover and shadow
        no_data_mask = layover.And(shadow).rename("no_data_mask")

        return no_data_mask

    # get the image geometry and projection
    geom = image.geometry(scale)
    proj = image.select(1).projection()
    angle_band = image.select("angle")

    # image to convert angle to radians
    to_radians = ee.Image.constant((math.pi / 180))
    # create a 90 degree image in radians
    ninetyRad = ee.Image.constant(90).multiply(to_radians)

    # calculate the look direction
    heading = (
        ee.Terrain.aspect(image.select("angle"))
        .reduceRegion(ee.Reducer.mean(), geom, scale)
        .get("aspect")
    )

    # the numbering follows the article chapters
    # 2.1.1 Radar geometry
    theta_iRad = image.select("angle").multiply(to_radians)
    phi_iRad = ee.Image.constant(heading).multiply(to_radians)

    # 2.1.2 Terrain geometry
    alpha_sRad = (
        ee.Terrain.slope(elevation)
        .select("slope")
        .multiply(to_radians)
        .setDefaultProjection(proj)
    )

    phi_sRad = (
        ee.Terrain.aspect(elevation)
        .select("aspect")
        .multiply(to_radians)
        .setDefaultProjection(proj)
    )

    # 2.1.3 Model geometry
    # reduce to 3 angle
    phi_rRad = phi_iRad.subtract(phi_sRad)

    # slope steepness in range (eq. 2)
    alpha_rRad = (alpha_sRad.tan().multiply(phi_rRad.cos())).atan()

    # slope steepness in azimuth (eq 3)
    alpha_azRad = (alpha_sRad.tan().multiply(phi_rRad.sin())).atan()

    # local incidence angle (eq. 4)
    theta_liaRad = (
        alpha_azRad.cos().multiply((theta_iRad.subtract(alpha_rRad)).cos())
    ).acos()
    theta_liaDeg = theta_liaRad.multiply(180 / math.pi)

    # 2.2
    # Gamma_nought
    gamma0 = image.divide(theta_iRad.cos())

    if model == "volume":
        scf = _volumetric_model_SCF(theta_iRad, alpha_rRad)

    elif model == "surface":
        scf = _surface_model_SCF(theta_iRad, alpha_rRad, alpha_azRad)

    else:
        raise NotImplementedError(
            f"Defined model, {model}, has not been implemented. Options are 'volume' or 'surface'"
        )

    # apply model for Gamm0_f
    gamma0_flat = gamma0.divide(scf)

    # calculate layover and shadow mask
    masks = _masking(alpha_rRad, theta_iRad, buffer)

    return (
        gamma0_flat.updateMask(masks)
        .addBands(angle_band)
        .addBands(theta_liaDeg.rename("local_inc_angle"))
    )

# define variables used within the slope_correction function
elevation = ee.Image("NASA/NASADEM_HGT/001").select("elevation")
model="volume"
buffer=100 # buffer areas of terrain shadow, in meters
scale=1000 # processing scale for reductions

In [ ]:
# define a function to apply the gamma map speckle filter algorithm
def gamma_map(image):
    """Gamma Map speckle filtering algorithm.
    Algorithm adapted from https://groups.google.com/g/google-earth-engine-developers/c/a9W0Nlrhoq0/m/tnGMC45jAgAJ.

    args:
        img (ee.Image): Earth engine image object. Expects that imagery is a SAR image in power scale

    returns:
        ee.Image: filtered SAR power image using the Gamma Map algorithm
    """

    img_bands = image.bandNames()

    # Square kernel, window should be odd (typically 3, 5 or 7)
    weights = ee.List.repeat(ee.List.repeat(1, window), window)
    midPt = (window // 2) + 1 if (window % 2) != 0 else window // 2

    # ~~(window/2) does integer division in JavaScript
    kernel = ee.Kernel.fixed(window, window, weights, midPt, midPt, False)

    # Get mean and variance
    mean = image.reduceNeighborhood(ee.Reducer.mean(), kernel)
    variance = image.reduceNeighborhood(ee.Reducer.variance(), kernel)

    # "Pure speckle" threshold
    ci = variance.sqrt().divide(mean)  # square root of inverse of enl

    # If ci <= cu, the kernel lies in a "pure speckle" area -> return simple mean
    cu = 1.0 / math.sqrt(enl)

    # If cu < ci < cmax the kernel lies in the low textured speckle area -> return the filtered value
    cmax = math.sqrt(2.0) * cu

    alpha = ee.Image(1.0 + cu * cu).divide(ci.multiply(ci).subtract(cu * cu))
    b = alpha.subtract(enl + 1.0)
    d = (
        mean.multiply(mean)
        .multiply(b)
        .multiply(b)
        .add(alpha.multiply(mean).multiply(image).multiply(4.0 * enl))
    )
    f = b.multiply(mean).add(d.sqrt()).divide(alpha.multiply(2.0))

    caster = ee.Dictionary.fromLists(
        img_bands, ee.List.repeat("float", img_bands.length())
    )
    img1 = (
        mean.updateMask(ci.lte(cu))
        .rename(img_bands)
        .cast(caster)
    )
    img2 = (
        f.updateMask(ci.gt(cu)).updateMask(ci.lt(cmax))
        .rename(img_bands)
        .cast(caster)
    )
    img3 = image.updateMask(ci.gte(cmax)).rename(img_bands).cast(caster)

    # If ci > cmax do not filter at all (i.e. we don't do anything, other then masking)
    output = (
        ee.ImageCollection([img1, img2, img3])
        .reduce(ee.Reducer.firstNonNull())
        .rename(img_bands)
        .clip(image.geometry(1e3))
    )

    # Compose a 3 band image with the mean filtered "pure speckle", the "low textured" filtered and the unfiltered portions
    return output
    

window=7 # filtering window size
enl=4.9 # equivalent number of looks, for S1 enl ≈ 5

In [ ]:
# this loads in a global vector file of countries
# filter by country of interest
region = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filter(
    ee.Filter.eq("country_na","Cambodia")
)

In [ ]:
# read in the power scale imagecollection
s1_power_asc = (
    ee.ImageCollection("COPERNICUS/S1_GRD_FLOAT")
    # filter for a flooding date
    .filterDate('2019-09-11', '2019-09-12')
    # filter for data in Cambodia
    .filterBounds(region)
    # Filter to get images with VV and VH dual polarization.
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
    # Filter to get images collected in interferometric wide swath mode.
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    # filter orbit pass
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
)

In [ ]:
# apply RTC to imagery
s1_rtc = s1_power_asc.map(slope_correction)
# apply speckle filter
s1_specfiltered = s1_rtc.map(gamma_map)
# convert to dB units
s1_db = s1_specfiltered.map(power_to_db)

In [ ]:
s1_mosaic = s1_db.mosaic()

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.centerObject(region,7)

Map.addLayer(s1_mosaic.reproject(ee.Projection("EPSG:4326").atScale(30)), {"bands":"VV", "min": -25, "max": 0}, 'dB image (reproject)')
Map.addLayer(s1_mosaic, {"bands":"VV", "min": -25, "max": 0}, 'dB image')

Map.addLayerControl()

Map

### Automated thresholding

Now that we have our image we can begin our processing to extract surface water information using Otsu’s threshold. As mentioned before, the Otsu’s thresholding algorithm is histogram based, therefore we will need to create a histogram of values.  Otsu’s method only works for grayscale imagery (i.e. using one band) so we will use the VV band from S1. Also, Earth Engine allows us to easily calculate a histogram using a reducer. We will demostrate extracting a histogram for the whole image.


In [ ]:
# specify band to use for Otsu thresholding
band = 'VV'

# define a reducer to calculate a histogram of values
histogram_reducer = ee.Reducer.histogram(255,0.1)

# reduce all of the image values
global_histogram = ee.Dictionary(
  s1_mosaic.select(band).reduceRegion(
    reducer = histogram_reducer,
    geometry = region.geometry(maxError=1e3),
    scale = 30,
    maxPixels = 1e10,
    bestEffort = True
  ).get(band)
)

In [ ]:
# extract out the histogram buckets and counts per bucket
bins = ee.List(global_histogram.get('bucketMeans')).getInfo()
counts = ee.List(global_histogram.get('histogram')).getInfo()

In [ ]:
fig, ax = plt.subplots()
ax.fill_between(bins,counts)
ax.set_xlabel("Backscatter [dB]")
ax.set_ylabel("Count")
ax.set_xlim(-30,10)
show()

As seen in the histogram, the data points are heavily skewed towards values around -10 to -5 dB. However, we can still see other small peaks of low backscatter values from -20 to -15 dB, these are the open water values.

Next we will apply the Otsu’s thresholding algorithm on the histogram we just calculated. Earth Engine does not have a function for Otsu’s method, however, we can create a function that calculates the optimal threshold:


In [ ]:
def otsu(histogram):
    """Otsu's method threhsolding algorithm.
    Computes single intensity threshold that separate histogram into two classes, foreground and background

    args:
        histogram (ee.Dictionary): computed object from ee.Reducer.histogram with keys "histogram" and "bucketMeans"

    returns:
        ee.Number: value of maximum inter-class intensity variance based on histogram
    """
    counts = ee.Array(ee.Dictionary(histogram).get("histogram"))
    means = ee.Array(ee.Dictionary(histogram).get("bucketMeans"))
    size = means.length().get([0])
    total = counts.reduce(ee.Reducer.sum(), [0]).get([0])
    sums = means.multiply(counts).reduce(ee.Reducer.sum(), [0]).get([0])
    mean = sums.divide(total)
    indices = ee.List.sequence(1, size)
    # Compute between sum of squares, where each mean partitions the data.

    def bss_function(i):
        aCounts = counts.slice(0, 0, i)
        aCount = aCounts.reduce(ee.Reducer.sum(), [0]).get([0])
        aMeans = means.slice(0, 0, i)
        aMean = (
            aMeans.multiply(aCounts)
            .reduce(ee.Reducer.sum(), [0])
            .get([0])
            .divide(aCount)
        )
        bCount = total.subtract(aCount)
        bMean = sums.subtract(aCount.multiply(aMean)).divide(bCount)
        return aCount.multiply(aMean.subtract(mean).pow(2)).add(
            bCount.multiply(bMean.subtract(mean).pow(2))
        )

    bss = indices.map(bss_function)
    output = means.sort(bss).get([-1])
    return ee.Number(output)

In [ ]:
# apply Otsu algorithm to get threshold value
global_threshold = otsu(global_histogram)

In [ ]:
global_t = global_threshold.getInfo()
print(f"Calculated threshold: {global_t:.4f}")

In [ ]:
fig, ax = plt.subplots()
ax.fill_between(bins,counts)
ax.vlines(global_t,0,max(counts),color="r")
ax.set_xlabel("Backscatter [dB]")
ax.set_ylabel("Count")
ax.set_xlim(-30,10)
show()

 We can now apply that threshold on the imagery and inspect how the extracted water looks compared to the original image. Here we apply the threshold and add the water image to the map:


In [ ]:
water_img = s1_mosaic.select(band).lt(global_threshold)

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.centerObject(region,7)

Map.addLayer(s1_mosaic.reproject(ee.Projection("EPSG:4326").atScale(30)), {"bands":"VV", "min": -25, "max": 0}, 'dB image')
Map.addLayer(water_img.selfMask(), {"min": 0, "max": 1, "palette":cmaps.get_palette("Blues")}, 'Water image')


Map.addLayerControl()

Map

From afar the results look promising, the blue areas shown overlap with the low backscatter (specular reflectance) that is representative of open water in C-band SAR imagery. However, upon closer inspection we can see that the extracted water overestimates in some areas that can be land areas.

We see an overestimation as large local errors may be introduced when calculating a constant threshold for distinguishing water and land when using an image-wide histogram. It is due to this issue that algorithms have been developed to constrain the histogram sampling and have a more locally contextual threshold.


### Adaptive Thresholding

As seen from the previous section, surface water usually constitutes only a small fraction of the overall land cover within an image. This makes it harder to apply threshold-based methods to extract water. The challenge is to establish a varying threshold that can be derived automatically. This section walks through an adaptive thresholding technique designed to overcome the challenges of using a global threshold.

The method we will discuss was developed by [Donchyts et al. (2016)](https://doi.org/10.3390/rs8050386) and applied to the modified normalized difference water index (MNDWI) from Landsat 8 imagery. The algorithm finds edges within the image, then buffers the areas of identified edges, and uses the buffered area to sample a histogram for Otsu thresholding. This approach assumes that the edges detected are from water. The result is a bimodal histogram from the area around water edges that can be used to calculate a refined threshold. This approach was refined by [Markert et al. (2020)](https://doi.org/10.3390/rs12152469) where the main change was instead on calculating the edges on the raw values (from an index or otherwise), an initial segmentation threshold is provided to create a binary image to alleviate any edges being defined from other classes that are present in imagery (i.e., Urban Areas or Forests). Then the defined edges are filtered by length to omit small edges that can occur and skew the histogram sampling. This requires a few parameters that can be tuned, namely the initial threshold, edge length, and buffer size. Here we define a few of those parameters:

In [ ]:
# define paramters for the adaptive thresholding 
initial_threshold = -16 # initial estimate of water/no-water for estimating the edges
connected_pixels = 100 # number of connected pixels to use for length calculation
edge_length = 20 # length of edges to be considered water edges
edge_buffer = 300 # buffer in meters to apply to edges
canny_threshold = 1 # threshold for canny edge detection
canny_sigma = 1 # sigma value for gaussian filter in canny edge detection
canny_lt = 0.05 # lower threshold for canny detection

With these parameters defined, we can begin the process of constraining the histogram sampling:

In [ ]:
# get preliminary water
binary = s1_mosaic.select(band).lt(initial_threshold).rename('binary');

# get projection information to convert buffer size to npixels
image_proj = s1_power_asc.first().select(band).projection()

# get canny edges
canny = ee.Algorithms.CannyEdgeDetector(binary, canny_threshold, canny_sigma)
# process canny edges
# get the edges and length of edges
connected  = canny.updateMask(canny).lt(canny_lt).connectedPixelCount(connected_pixels, True)
# mask short edges that can be noise
edges = connected.gte(edge_length)
# calculate the buffer in pixel size
edge_buffer_pixel = ee.Number(edge_buffer).divide(image_proj.nominalScale())
# buffer the edges using an efficient dilation operation
buffered_edges = edges.fastDistanceTransform().lt(edge_buffer_pixel)

# mask areas not within buffer 
edge_image = s1_mosaic.select(band).updateMask(buffered_edges)


Now that we have the edge information and data to sample processed, we can visually inspect what the algorithm is doing. Here we will display the edges calculated as well as the buffered edges to highlight which data is being sampled:


In [ ]:
# Visualize the results
Map = geemap.Map()

Map.centerObject(region,7)

Map.addLayer(s1_mosaic.reproject(ee.Projection("EPSG:4326").atScale(30)), {"bands":"VV", "min": -25, "max": 0}, 'dB image')
Map.addLayer(edges,{"palette":"red"},"Detected water edges")
Map.addLayer(buffered_edges,{"palette":"gray,yellow","min":0, "max":1,"opacity":0.5},"Buffered water edges")
Map.addLayer(edge_image, {"bands":"VV", "min": -25, "max": 0}, 'dB image edges')


Map.addLayerControl()

Map

From this point we have our regions that we want to sample that are more representative of a bimodal histogram and we have masked out areas that we don’t want to sample. Now we can calculate the histogram as before and make a plot:

In [ ]:
# reduce all of the image values
local_histogram = ee.Dictionary(
  edge_image.select(band).reduceRegion(
    reducer = histogram_reducer,
    geometry = region.geometry(1e3),
    scale = 30,
    maxPixels = 1e10,
    bestEffort = True
  ).get(band)
)

# apply Otsu algorithm to get threshold value
local_threshold = otsu(local_histogram)

In [40]:
# extract out the histogram buckets and counts per bucket
bins = ee.List(local_histogram.get('bucketMeans')).getInfo()
counts = ee.List(local_histogram.get('histogram')).getInfo()
local_t = local_threshold.getInfo()

print(f"Calculated threshold: {local_t:.4f}")

Calculated threshold: -14.6020


In [ ]:
fig, ax = plt.subplots()
ax.fill_between(bins,counts)
ax.vlines(local_t,0,max(counts),color="r")
ax.set_xlabel("Backscatter [dB]")
ax.set_ylabel("Count")
ax.set_xlim(-30,10)
show()

We can see from the histogram that we have a better distinction of water/land values which meet our assumption of Otsu’s thresholding of two classes. This allows the algorithm to more accurately calculate the threshold for water. The last thing left to do is apply the calculated adaptive threshold on the imagery and add it to the map:

In [ ]:
water_img = s1_mosaic.select(band).lt(local_threshold)

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.centerObject(region,7)

Map.addLayer(s1_mosaic.reproject(ee.Projection("EPSG:4326").atScale(30)), {"bands":"VV", "min": -25, "max": 0}, 'dB image')
Map.addLayer(water_img.selfMask(), {"min": 0, "max": 1, "palette":cmaps.get_palette("Blues")}, 'Water image')


Map.addLayerControl()

Map

Now that we have a surface water map and we are moderately confident it represents the actual surface water for that day, we can begin to identify flooded areas by differencing our resulting map with historical information.

## Extracting flood area

Up to this point we have been mapping surface water, surface water is a term that includes permanent and seasonal water for what was observed by the sensor. What we need to do now is to identify areas from our image that are considered permanent water. There are typically two approaches to map flooded areas with a thematic surface water map: 1) comparing pre- and post-event images to estimate changes or 2) compare extracted surface water with historically observed permanent water.

To achieve the goal of flood mapping, we will use the historical JRC Global Surface Water dataset ([Pekel et al., 2016](https://doi.org/10.1038/nature20584)) to define permanent water and then find the difference to extract flooded areas. We already have our surface water map post-event, now we need to define the JRC data:


In [ ]:
# get the previous 5 years of permanent water
# get the JRC historical yearly dataset
jrc = (
    ee.ImageCollection("JRC/GSW1_2/YearlyHistory") 
    .filterDate("1985-01-01","2019-09-12") # filter for historical data up to date of interest
    .limit(5, "system:time_start", False) # grab the 5 latest images/years
)

This data is a yearly classification of permanent and seasonal water, so now what we need to do is reclassify the imagery to just permanent water:

In [ ]:
# define function to extract out the water class
def permanent_water_reclassify(image):
    return image.select("waterClass").eq(3)

permanent_water = (
    jrc
    .map(permanent_water_reclassify) # apply reclassification
    .sum() # reduce collection to information on if a pixel has been classified as permanent water in the past 5 years
    .unmask(0) # make sure we have a value everywhere
    .gt(0) # get a binary image of 1 if permanent water in the past 5 years, otherwise 0
    .updateMask(water_img.mask()) # mask for only the water image we just calculated
)

In [ ]:
# find areas where there is not permanent water and water from observation
flood_img = permanent_water.Not().And(water_img)

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.centerObject(region,7)

Map.addLayer(s1_mosaic.reproject(ee.Projection("EPSG:4326").atScale(30)), {"bands":"VV", "min": -25, "max": 0}, 'dB image')
Map.addLayer(water_img.selfMask(), {"min": 0, "max": 1, "palette":cmaps.get_palette("Blues")}, 'Observed Surface Water')
# add the permanent water layer to the map
Map.addLayer(permanent_water.selfMask(), {"palette": "royalblue"}, "JRC permanent water");
# add flood image to map
Map.addLayer(flood_img.selfMask(),{"palette":"firebrick"},"Flood areas");

Map.addLayerControl()

Map

There are nuances associated with comparing optically derived water information (like from JRC) with SAR water maps. For example, any surface that is large enough and smooth can “look” like water in SAR imagery because of specular scattering and can be wrongly classified as floods. Examples of this are airports, exposed channel beds, and highways. Therefore, comparing pre- and post-event imagery from the same sensor is best, however, it is challenging to define events in seasonal flooding (such as this case) making a pre- and post-event comparison a little more complicated.
